<h1> Manhattan Bakery Study </h1>

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

Libraries imported.


<h2> Download and Prepare the Data </h2>

In [8]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [10]:
neighborhoods_data = newyork_data['features']

In [11]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)


In [12]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

<h2> Data Exploration </h2>

In [13]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [16]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [17]:
CLIENT_ID = 'DMCR2FZPXYOKLWAMLUYHYZPH4FCFNJLPQSTKBO3S4C2GUDJ1' 
CLIENT_SECRET = 'W43OVT0FPMZNADZCIUQWFLHKLU1JGVU3YLQI45DIIEOS5B53' 
VERSION = '20180605' 

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] 
neighborhood_longitude = manhattan_data.loc[0, 'Longitude']
neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] 



In [22]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [24]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [25]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3158, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [26]:
bakerys_df = manhattan_venues[manhattan_venues['Venue Category'] == 'Bakery'].reset_index(drop=True)
bakerys_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,Michaeli Bakery,40.714704,-73.991847,Bakery
1,Chinatown,40.715618,-73.994279,Fay Da Bakery,40.716765,-73.997874,Bakery
2,Chinatown,40.715618,-73.994279,La Bella Ferrara,40.717450,-73.998373,Bakery
3,Chinatown,40.715618,-73.994279,Mei Li Wah,40.715520,-73.997973,Bakery
4,Washington Heights,40.851903,-73.936900,181st St Bakery & Deli,40.851014,-73.938592,Bakery
5,Washington Heights,40.851903,-73.936900,Fort Washington Bakery & Deli,40.851046,-73.938645,Bakery
6,Washington Heights,40.851903,-73.936900,El Panadero Bakery,40.847987,-73.934776,Bakery
7,Washington Heights,40.851903,-73.936900,Floridita Broadway Bakery,40.847781,-73.937851,Bakery
8,Inwood,40.867684,-73.921210,ChocNYC,40.868396,-73.918742,Bakery
9,Inwood,40.867684,-73.921210,Bizcocho De Colores,40.866035,-73.919396,Bakery


In [27]:
bakerys_df.shape

(71, 7)

There are 71 bakeries in Manhattan. Let's dive deeper to find the Neighborhood with the most.

In [32]:
count = bakerys_df.groupby('Neighborhood').count()
count

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Carnegie Hill,3,3,3,3,3,3
Chelsea,3,3,3,3,3,3
Chinatown,4,4,4,4,4,4
Civic Center,1,1,1,1,1,1
Clinton,1,1,1,1,1,1
East Harlem,4,4,4,4,4,4
East Village,2,2,2,2,2,2
Financial District,1,1,1,1,1,1
Flatiron,2,2,2,2,2,2


In [33]:
count.sort_values(by='Venue Category',ascending=False,inplace=True)

In [34]:
count

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Little Italy,6,6,6,6,6,6
Chinatown,4,4,4,4,4,4
West Village,4,4,4,4,4,4
Washington Heights,4,4,4,4,4,4
East Harlem,4,4,4,4,4,4
Upper East Side,4,4,4,4,4,4
Carnegie Hill,3,3,3,3,3,3
Upper West Side,3,3,3,3,3,3
Soho,3,3,3,3,3,3


Little Italy, Chinatown, West village, Washington Hieghts, East Harlem and Upper East Side have the most bakeries.

Now we have to use k-means to cluster the Neighborhoods and find the ones similar to these.

<h2> Cluster Analysis </h2>

In [35]:
manhattan_venues.groupby('Neighborhood').count()

manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

In [36]:
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Res

In [37]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [41]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 2, 1, 2, 1, 2, 1, 0, 1, 1], dtype=int32)

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Memorial Site,Coffee Shop,Shopping Mall,Mexican Restaurant,Wine Shop,Pizza Place,Playground
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Bookstore,Wine Shop,Japanese Restaurant,Bakery,Cosmetics Shop,Pizza Place,Italian Restaurant
2,Central Harlem,Seafood Restaurant,Gym / Fitness Center,African Restaurant,American Restaurant,Bar,French Restaurant,Chinese Restaurant,Caribbean Restaurant,Gym,Market
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Cocktail Bar,Pizza Place,Pet Store,Nightclub
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,American Restaurant,Salon / Barbershop,Optical Shop,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant


In [44]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,Bakery,American Restaurant,Salon / Barbershop,Optical Shop,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Grocery Store,Bakery,Chinese Restaurant,Mobile Phone Shop,Pizza Place,Latin American Restaurant,Mexican Restaurant,Bank,Spanish Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Café,Pizza Place,Restaurant,Frozen Yogurt Shop,Park,Chinese Restaurant,Bakery,American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Bakery,Sushi Restaurant,Cocktail Bar,Sandwich Place,Yoga Studio


In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
manhattan_merged.set_index('Neighborhood', inplace= True)
manhattan_merged.loc[['Little Italy', 'Chinatown', 'West Village'],['Cluster Labels']]

KeyError: "None of ['Neighborhood'] are in the columns"

In [63]:
manhattan_merged.loc[['Little Italy', 'Chinatown', 'West Village'],['Cluster Labels']]

,Cluster Labels
Neighborhood,
Little Italy,1
Chinatown,1
West Village,2


In [80]:
options = manhattan_merged[manhattan_merged['Cluster Labels'] == 1]

In [81]:
options

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,,,,
Chinatown,Manhattan,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,Bakery,American Restaurant,Salon / Barbershop,Optical Shop,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant
Central Harlem,Manhattan,40.815976,-73.943211,1,Seafood Restaurant,Gym / Fitness Center,African Restaurant,American Restaurant,Bar,French Restaurant,Chinese Restaurant,Caribbean Restaurant,Gym,Market
Clinton,Manhattan,40.759101,-73.996119,1,Theater,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Spa,Wine Shop,Cocktail Bar,Thai Restaurant,American Restaurant,Sandwich Place
Midtown,Manhattan,40.754691,-73.981669,1,Coffee Shop,Hotel,Steakhouse,Sporting Goods Shop,Clothing Store,Spa,Café,Sandwich Place,Bakery,Theater
Murray Hill,Manhattan,40.748303,-73.978332,1,Sandwich Place,Hotel,Coffee Shop,Sushi Restaurant,Bar,Japanese Restaurant,Burger Joint,Gym / Fitness Center,American Restaurant,Cuban Restaurant
East Village,Manhattan,40.727847,-73.982226,1,Bar,Pizza Place,Coffee Shop,Wine Bar,Mexican Restaurant,Ice Cream Shop,Korean Restaurant,Speakeasy,Cocktail Bar,Ramen Restaurant
Lower East Side,Manhattan,40.717807,-73.980890,1,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Art Gallery,Ramen Restaurant,Japanese Restaurant,Café,Clothing Store,Flower Shop
Little Italy,Manhattan,40.719324,-73.997305,1,Bakery,Chinese Restaurant,Café,Bubble Tea Shop,Mediterranean Restaurant,Italian Restaurant,Hotpot Restaurant,Hotel,Thai Restaurant,Sandwich Place
Gramercy,Manhattan,40.737210,-73.981376,1,Italian Restaurant,Coffee Shop,Bar,American Restaurant,Pizza Place,Grocery Store,Mexican Restaurant,Ice Cream Shop,Playground,Cocktail Bar


In [82]:
options = options.merge(count, how='inner', on= 'Neighborhood')

In [83]:
options = options[options['Venue'] < 3]

options

In [84]:
options

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,,,,,,,,,,,,,,
Clinton,Manhattan,40.759101,-73.996119,1,Theater,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Spa,Wine Shop,Cocktail Bar,Thai Restaurant,American Restaurant,Sandwich Place,1,1,1,1,1,1
Murray Hill,Manhattan,40.748303,-73.978332,1,Sandwich Place,Hotel,Coffee Shop,Sushi Restaurant,Bar,Japanese Restaurant,Burger Joint,Gym / Fitness Center,American Restaurant,Cuban Restaurant,1,1,1,1,1,1
East Village,Manhattan,40.727847,-73.982226,1,Bar,Pizza Place,Coffee Shop,Wine Bar,Mexican Restaurant,Ice Cream Shop,Korean Restaurant,Speakeasy,Cocktail Bar,Ramen Restaurant,2,2,2,2,2,2
Lower East Side,Manhattan,40.717807,-73.980890,1,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Art Gallery,Ramen Restaurant,Japanese Restaurant,Café,Clothing Store,Flower Shop,2,2,2,2,2,2
Financial District,Manhattan,40.707107,-74.010665,1,Coffee Shop,Pizza Place,Hotel,Cocktail Bar,Mexican Restaurant,Bar,Park,Sandwich Place,American Restaurant,Japanese Restaurant,1,1,1,1,1,1
Midtown South,Manhattan,40.748510,-73.988713,1,Korean Restaurant,Hotel,Japanese Restaurant,Dessert Shop,Bar,Burger Joint,American Restaurant,Gym / Fitness Center,Clothing Store,Salad Place,1,1,1,1,1,1
Flatiron,Manhattan,40.739673,-73.990947,1,Gym / Fitness Center,New American Restaurant,Café,Yoga Studio,Vegetarian / Vegan Restaurant,Italian Restaurant,Gym,Mediterranean Restaurant,Furniture / Home Store,Cycle Studio,2,2,2,2,2,2


Given the Cluster analysis the options for oppening a Bakery are: Clinton, Murray Hill, East Village, Lower East Side, Financial District, Midtown South and Flatiron.